In [113]:
import numpy as np
import pandas as pd
import time
import wrds
from matplotlib import pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from scipy.interpolate import UnivariateSpline, splev, splrep
from scipy.optimize import minimize
from math import *
np.random.seed(6078)
import warnings
warnings.filterwarnings('ignore')
# from dask.multiprocessing import get
# import dask.dataframe as dd

## Options Distribution Estimation Model

Data Comes from the merged_data.csv generated by combining all options, with the earnings dates from the time sereis feature selection outputs. Then filtered down to just options priced 1 week prior to earnings. These options will be used to estimate the post earnings drift

In [114]:
opm = pd.read_csv('merged_data.csv', nrows=200000)
#opm['date'] = pd.to_datetime(opm['date'], format='%Y-%m-%d')
#opm['exdate'] = pd.to_datetime(opm['exdate'], format='%Y-%m-%d')
# opm = opm[(opm['exdate'] - opm['date'] == np.timedelta64(-8, 'D'))] # Use to check if dates were matched correctly
opm.rename(columns={'strike_price': 'STRIKE'}, inplace=True)
opm['STRIKE'] = opm['STRIKE']/1000 # Strike comes expressed in par value, needs to be converted to dollars

The options are grouped by date and secid, relevant features as of now are the opprc (price), strike and cp_flag, moneyness, implied vol and spread are likely to be incorporated in a final version

In [115]:
opm.head()

,secid,exdate,date,cp_flag,STRIKE,volume,open_interest,impl_volatility,opprc,moneyness,tte,close,spread,noi
0,210354,20190719,20190711,C,14.0,0,44,1.651489,8.550,0.622222,8,22.5,0.50,0
1,210354,20190719,20190711,C,18.0,1,10,0.859370,4.550,0.800000,8,22.5,0.10,0
2,210354,20190719,20190711,C,19.0,0,3,0.814534,3.600,0.844444,8,22.5,0.20,0
3,210354,20190719,20190711,C,20.0,0,39,0.419988,2.525,0.888889,8,22.5,0.35,0
4,210354,20190719,20190711,C,21.0,14,396,0.651550,1.800,0.933333,8,22.5,0.10,-40


Options are cleaned using no arbitrage arguments to prevent bad data. Calls must be monotonically decreasing with strike, and puts must be monotonically increasing with strike. The calls and puts are combined to make straddle prices

In [117]:
def clean_fairs(chain):
    chain = chain.sort_values(["cp_flag", "STRIKE"]).reset_index()
    phain = chain[chain['cp_flag'] == 'P']
    chain = chain[chain['cp_flag'] == 'C']
    chain['CV'] = -((-chain['opprc']).cummax()) # forces decreasing
    phain['PV'] = ((phain['opprc']).cummax()) # forces increasing
    chain = chain[["STRIKE", "CV"]]
    phain = phain[["STRIKE", "PV"]]
    df = chain.merge(phain, on='STRIKE') 
    df["V"] = df["CV"] + df["PV"] # straddle calculation
    return df

In [118]:
def implied_probs(chain, s=0.1, M=1000):
    df = clean_fairs(chain) # Clean selected options chain
    df =  df.drop_duplicates(subset='STRIKE', keep="first").reset_index() # Handles case where strikes are repeated
    if len(df) < 5: # The distribution can not be accurately generated if there are fewer than 5 strikes
        return np.nan
    spl = UnivariateSpline(df['STRIKE'], df['V'], k=4, s=s*df.shape[0]) # Spline is fit with smoothing on straddle
    df['Vhat'] = spl(df['STRIKE']) # Estimate for the straddle value
    df['IP'] = np.maximum(spl.derivative().derivative()(df['STRIKE']), 0) # Probability is second derivative over strikes
    df["PCDF"] = df["IP"].cumsum() # Convert probabiility into CDF
    df["PCDF"] = df["PCDF"] / df["PCDF"].max() # Normalization step
    return df

Once the distribution is generated there are many options for how to incorporate this data into a trading stratagey. The current implementation is to determine the implied median for the post earnings drift, this signals the skew of the distribution.

In [119]:
def implied_median(chain):
    df = implied_probs(chain) # Generate implied probabilities
    try:
        mid_idx = df[(df['PCDF'] > 0.5)].index[0] # Locate the median
        lb = df.loc[mid_idx-1, 'STRIKE']
        lbp = df.loc[mid_idx-1, 'PCDF']
        ub = df.loc[mid_idx, 'STRIKE']
    except:
        return np.nan # If there is a bad median due to low sample size, return null
    
    ubp = df.loc[mid_idx, 'PCDF']
    slope = (ubp - lbp) / (ub - lb) # Linear approximation of median between nearest values
    res = (0.5 - lbp) / slope + lb
    return res

def implied_move(chain):
    return implied_median(chain) / chain.PRC.max() - 1 # Returns the drift signald by the median

In [124]:
# Options chains are isolated by secid and the expiry (friday after earnings)
opm.groupby(["secid", "exdate"]).apply(implied_median) # Running the median estimation on isolated options chains

secid   exdate  
8170    20180615          NaN
        20190215          NaN
        20191115          NaN
100862  20181221          NaN
        20191220          NaN
                      ...    
213731  20190906    91.806615
213761  20191115          NaN
213776  20190920    28.744683
        20191213    24.137452
213814  20191115          NaN
Length: 2541, dtype: float64

In [126]:
opm_app_mean = opm.groupby(["secid", "exdate"]).apply(implied_median)

In [127]:
opm_app_mean.to_csv("opm_app_mean.csv") # Writing data output to csv